In [1]:
# import libraries 

from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import numpy as np

In [2]:
#Extract the Top 1000 movies URL

url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'

#request allow you to send HTTP request

response = requests.get(url)


In [3]:
# Check the response status: 200 means ok, 503: error, un-available
response

<Response [200]>

In [4]:
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
type(soup)

bs4.BeautifulSoup

In [6]:
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>IMDb "Top 1000"
(Sorted by IMDb Rating Descending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/title/?groups=top_1000" rel="canonical"/>
<meta content="http://www.imdb.com/search/title/?groups=top_1000" property=

In [7]:
# create empty list for the columns to be extracted 
movie_name = []
year = []
time = []
rating = []
metascore = []
votes = []
gross = []
description = []
Director = []
Stars = []

In [8]:
#storing required data fields in the variable

movie_data = soup.findAll('div', attrs= {'class': 'lister-item mode-advanced'})


In [9]:
type(movie_data)

bs4.element.ResultSet

In [10]:
#calling one by one using for loop

for store in movie_data:
    name = store.h3.a.text
    movie_name.append(name)
    
    #RELEASE_YEAR
    year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
    year.append(year_of_release)

    #RUNTIME
    runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
    time.append(runtime)
    
    #RATINGS
    rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    rating.append(rate)
    
    #METASCORE- If Values are not found, default to Zero (0)
    meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '0'
    metascore.append(meta)
    
    #since, gross and votes have same attributes, that's why we had created a common variable and then used indexing
    
    value = store.find_all('span', attrs = {'name': 'nv'})
    
    #VOTES
    vote = value[0].text
    votes.append(vote)
    
    #GROSS- If Values are not found, default to Zero (0)
    grosses = value[1].text if len(value) >1 else '0'
    gross.append(grosses)
    
    # Description of the Movies - If Description is not found, default to NULL
    describe = store.find_all('p', class_ = 'text-muted')
    description_ = describe[1].text.replace('\n', '') if len(describe) >1 else 'NULL'
    description.append(description_)
    
    #Cast Details -- Scraping Director name and Stars
    cast = store.find("p", class_ = '')
    
    #cast=cast.text
    #cast=cast.replace('\n','')
    
    cast = cast.text.replace('\n', '').split('|')
    
    cast = [x.strip() for x in cast]

    cast = [cast[i].replace(j, "") for i,j in enumerate(["Director:", "Stars:"])]

    # DIRECTOR NAMES
    
    Director.append(cast[0])
    
    #STARS NAMES
    Stars.append([x.strip() for x in cast[1].split(",")])
    
    

In [11]:
##Directors Name
## cast[0].split(':')[1]

In [12]:
##Stars Name
##cast[1].split(':')[1]

In [13]:

#creating a dataframe 
movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Year of relase': year, 'Watchtime': time, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross, 'Description': description, "Director": Director, 'Star': Stars})


In [27]:
movie_DF.head()

,Name of movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description,Director,Star
0,The Shawshank Redemption,1994,142,9.3,82,"2,778,886",$28.34M,"Over the course of several years, two convicts...",Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,175,9.2,100,"1,934,725",$134.97M,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,The Dark Knight,2008,152,9.0,84,"2,756,129",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,Schindler's List,1993,195,9.0,95,"1,398,555",$96.90M,"In German-occupied Poland during World War II,...",Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car..."
4,12 Angry Men,1957,96,9.0,97,"824,775",$4.36M,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."


In [26]:
movie_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name of movie     100 non-null    object
 1   Year of relase    100 non-null    object
 2   Watchtime         100 non-null    object
 3   Movie Rating      100 non-null    object
 4   Metascore         100 non-null    object
 5   Votes             100 non-null    object
 6   Gross collection  100 non-null    object
 7   Description       100 non-null    object
 8   Director          100 non-null    object
 9   Star              100 non-null    object
dtypes: object(10)
memory usage: 7.9+ KB


In [15]:
movie_DF.head()

,Name of movie,Year of relase,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description,Director,Star
0,The Shawshank Redemption,1994,142,9.3,82,"2,778,886",$28.34M,"Over the course of several years, two convicts...",Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,1972,175,9.2,100,"1,934,725",$134.97M,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,The Dark Knight,2008,152,9.0,84,"2,756,129",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,Schindler's List,1993,195,9.0,95,"1,398,555",$96.90M,"In German-occupied Poland during World War II,...",Steven Spielberg,"[Liam Neeson, Ralph Fiennes, Ben Kingsley, Car..."
4,12 Angry Men,1957,96,9.0,97,"824,775",$4.36M,The jury in a New York City murder trial is fr...,Sidney Lumet,"[Henry Fonda, Lee J. Cobb, Martin Balsam, John..."


In [16]:
movie_DF.count()

Name of movie       100
Year of relase      100
Watchtime           100
Movie Rating        100
Metascore           100
Votes               100
Gross collection    100
Description         100
Director            100
Star                100
dtype: int64

In [23]:
movie_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name of movie     100 non-null    object
 1   Year of relase    100 non-null    object
 2   Watchtime         100 non-null    object
 3   Movie Rating      100 non-null    object
 4   Metascore         100 non-null    object
 5   Votes             100 non-null    object
 6   Gross collection  100 non-null    object
 7   Description       100 non-null    object
 8   Director          100 non-null    object
 9   Star              100 non-null    object
dtypes: object(10)
memory usage: 7.9+ KB


In [29]:
#Saving data in csv file:

movie_DF.to_csv("Top_100_IMDB_Movies.csv",index=False)